# COMP9318 Project
## Exploratory Data Analysis

In [7]:
import helper as h
import pandas as pd
import sklearn

Read in word data:

In [8]:
data_loc = 'asset/training_data.txt'
words = h.get_words(data_loc)